In this notebook, we present various usecases to interact with ptype for column type predictions.

In [ ]:
# Preamble to run notebook in context of source package.
# NBVAL_IGNORE_OUTPUT
import sys
sys.path.insert(0, '../')

In [ ]:
from IPython.core.display import display, HTML
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcdefaults()
import numpy as np
import pandas as pd
# import pprint

from ptype.Column import Column2ARFF
from ptype.Ptype import Ptype
from ptype.utils import evaluate_types
from utils import *

### UCI Automobile Dataset

In [ ]:
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]

df = pd.read_csv('../data/auto.csv', names = headers)
df.head()

### The Analytical Task

This dataset is commonly used for a regression task, where the goal is to predict the price of an automobile given its attributes.

### A Solution using Standard Python Libraries
Let's now develop a simple solution for this problem. The solution is inspired from Kaggle (see https://www.kaggle.com/fazilbtopal/data-wrangling and https://www.kaggle.com/fazilbtopal/model-development-and-evaluation-with-python).

In [ ]:
features = ['horsepower', 'curb-weight', 'engine-size', 'highway-mpg']
target = ['price']

X = df[features]
y = df[target]

df = df[features+target]
df.head()

In [ ]:
# to see the error message, uncomment the following

lm = LinearRegression()
# lm.fit(X, y)
# y_hat = lm.predict(X)

We notice that some data entries are valued ? and that this leads Pandas to misclassify two data columns as object rather than int64.

In [ ]:
df.dtypes

In [ ]:
df.loc[130]

We need to "clean" the horsepower and price columns in terms of missing values. Let's first have a look at what we can do without ptype:

In [ ]:
# replace missing data encoding
df['horsepower'].replace("?", np.nan, inplace = True)
df['price'].replace("?", np.nan, inplace = True)

# check counts of missing data
missing_data = df.isnull().sum()
missing_data.sort_values(inplace=True, ascending=False)
display(missing_data.head())

# drop rows
n = df.shape[0]
df.dropna(subset=["horsepower", "price"], axis=0, inplace=True)
print("# rows deleted = " + str(n-df.shape[0]))

# update the indices
df.reset_index(drop=True, inplace=True)

In [ ]:
df.dtypes

In [ ]:
X = df[features].astype(float)
y = df[target].astype(float).values

lm.fit(X, y)
y_hat = lm.predict(X)

scatter_plot(y, y_hat)

In [ ]:
print('Mean Absolute Error:', round(metrics.mean_absolute_error(y, y_hat), 2))
print('Mean Squared Error:', round(metrics.mean_squared_error(y, y_hat), 2))
print('Root Mean Squared Error:', round(np.sqrt(metrics.mean_squared_error(y, y_hat)), 2))

In [ ]:
from pandas.io.json import build_table_schema

build_table_schema(df)

Let's now revisit the problem and see how we can use ptype to resolve it.

In [ ]:
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]

df = pd.read_csv('../data/auto.csv', names = headers, dtype='str')
df = df[features+target]
df.head()

ptype = Ptype()

In [ ]:
schema = ptype.fit_schema(df)
ptype.show_schema()

In [ ]:
df = ptype.transform_schema(df, schema)

In [ ]:
df.loc[130]

In [ ]:
df.dtypes

In [ ]:
# check counts of missing data
missing_data = df.isnull().sum()
missing_data.sort_values(inplace=True, ascending=False)
display(missing_data.head())

# drop rows
n = df.shape[0]
df.dropna(subset=["horsepower", "price"], axis=0, inplace=True)
print("# rows deleted = " + str(n-df.shape[0]))

# update the indices
df.reset_index(drop=True, inplace=True)

In [ ]:
X = df[features]
y = df[target].values

lm = LinearRegression()
lm.fit(X, y)
y_hat = lm.predict(X)

scatter_plot(y, y_hat)